In [ ]:
%matplotlib widget
import json
import os
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import Lib
import bqplot
from traitlets import Unicode, List, Instance, link, HasTraits
from IPython.display import display, clear_output, HTML, Javascript

In [ ]:
path='../monaco-data/output'
#fileName='model_values_cube.json'
fileName='exposures_cube.json'
fullName=path+'/'+fileName
cube=Lib.LoadCube(fullName)
cashflowsFileName='cashflows.json'
fullName=path+'/'+cashflowsFileName
f = open(fullName,)
cashflows=json.load(f)

exerciseFiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith(' (exercise).json')]

instruments=cube['time_series_names']

idx={}
idx['ins']=0
idx['scenario']=0

In [ ]:
def OnScenario(scenario):
    #print('OnScenario: '+str(scenario))
    idx['scenario']=scenario
    UpdateWidgets()

In [ ]:
def OnSelect(instrument):
    #print('OnSelect: '+instrument)
    idx['ins']=instruments.index(instrument)
    UpdateWidgets()

In [ ]:
def UpdateWidgets():
    #print('Update widgets: '+str(idx['ins'])+'/'+str(idx['scenario']))
    #Diffusion plot
    date_start=0
    date_end=len(cube['dates'])-1
    scenario_start=0
    scenario_end=cube['num_scenarios']-1
    
    data=[]
    for s in range(scenario_start,scenario_end+1):
        scenario_data=Lib.GetScenarioData(cube,s)
        data.append(scenario_data[:,idx['ins']])

    lines.x=cube['dates']
    lines.y=data
    
    #Summary table
    data=[]
    for d in range(date_start,date_end+1):
        vec=np.array([])
        for s in range(scenario_start,scenario_end+1):
            val=Lib.GetValue(cube,d,s,idx['ins'])
            vec=np.append(vec,val)
        avg=vec.mean()
        std=vec.std()
        data.append([d,d*0.25,avg,std])

    res=pd.DataFrame(data, columns=["Date #", "Date", "Average", "Standard deviation"]).round(2)
    table.value = res.to_html(escape=False)
    
    #PV and cashflows
    data=np.array(cashflows[idx['ins']][idx['scenario']])
    positive=data[:,1]>0
    c=list(map(Map,positive))

    scenario_data=Lib.GetScenarioData(cube,idx['scenario'])
    pv.x=cube['dates']
    pv.y=scenario_data[:,idx['ins']]

    bar.x=data[:,0]
    bar.y=data[:,1]
    
    #Exercise proababilities
    file=instruments[idx['ins']]+' (exercise).json'
    pcube=Lib.LoadCube(path+'/'+file,verbose=False)
    date_start=0
    date_end=len(pcube['dates'])-1
    scenario_start=0
    scenario_end=pcube['num_scenarios']-1
    data=[]
    for d in range(date_start,date_end+1):
        vec=np.array([])
        for s in range(scenario_start,scenario_end+1):
            val=Lib.GetValue(pcube,d,s,0)
            vec=np.append(vec,val)
        avg=vec.mean()
        cumulative=avg
        if d>0:
            cumulative+=data[d-1][3]
        data.append([d,d*0.25,avg,cumulative])
    data=np.array(data)
    cumProb.x=data[:,1]
    cumProb.y=data[:,3]
    prob.x=data[:,1]
    prob.y=data[:,2]

In [ ]:
from bqplot import (OrdinalScale, LinearScale, Bars, Lines,
                    Figure, Axis, ColorScale, ColorAxis, CATEGORY10)
scenario_start=0
scenario_end=cube['num_scenarios']-1

x_ord = LinearScale()
y_sc = LinearScale()

data=[]
for s in range(scenario_start,scenario_end+1):
    scenario_data=Lib.GetScenarioData(cube,s)
    data.append(scenario_data[:,0])

lines = Lines(x=cube['dates'],
              y=data,
              scales={'x': x_ord, 'y': y_sc},
              stroke_width=1)
ax_x = Axis(scale=x_ord, tick_rotate=0, tick_style={'font-size': 10})
ax_y = Axis(scale=y_sc, tick_format='0.2f', orientation='vertical', tick_style={'font-size': 8})

Figure(marks=[lines], axes=[ax_x, ax_y], animation_duration=500)

In [ ]:
table = widgets.HTML(
    value="Placeholder"
)
display(table)

In [ ]:
def Map(val):
    if val==True:
        return 'b'
    else:
        return 'r'

In [ ]:
c = 'rgbkymc'
data=np.array(cashflows[idx['ins']][idx['scenario']])
positive=data[:,1]>0
c=list(map(Map,positive))

x_cf_sc = LinearScale()
y_cf_sc = LinearScale()

pv=Lines(  x=[1,2,3],
           y=[1,1,1],
           scales={'x': x_cf_sc, 'y': y_cf_sc},
           colors=['red'],
           stroke_width=3)

bar=Bars(  x=[1,2,3],
           y=[2,2,2],
           scales={'x': x_cf_sc, 'y': y_cf_sc},
           stroke_width=1)

ax_cf_x = Axis(scale=x_cf_sc, tick_rotate=0, tick_style={'font-size': 8})
ax_cf_y = Axis(scale=y_cf_sc, tick_format='0.2f', orientation='vertical',tick_style={'font-size': 10})

Figure(marks=[bar,pv], axes=[ax_cf_x, ax_cf_y], animation_duration=50)

In [ ]:
x_pb_sc = LinearScale()
y_pb_sc = LinearScale()

cumProb=Lines(x=[1,2,3],
           y=[1,1,1],
           scales={'x': x_pb_sc, 'y': y_pb_sc},
           colors=['red'],
           stroke_width=3)

prob=Bars(x=[1,2,3],
           y=[2,2,2],
           scales={'x': x_pb_sc, 'y': y_pb_sc},
           stroke_width=1)

ax_pb_x = Axis(scale=x_pb_sc, tick_rotate=0, tick_style={'font-size': 8})
ax_pb_y = Axis(scale=y_pb_sc, tick_format='0.2f', orientation='vertical',tick_style={'font-size': 10})

Figure(marks=[cumProb,prob], axes=[ax_pb_x, ax_pb_y], animation_duration=50)

In [ ]:
scenario_slider = widgets.interact(OnScenario, scenario=(0, cube['num_scenarios']-1, 1))

In [ ]:
ins_picker = widgets.interact(OnSelect, instrument=list(instruments))